In [1]:
import MeCab

def get_iobcorpus(file_name):
    

    # アノテーション情報用意
    tag_number = []
    tag_label = []
    tag_vocab = {}
    with open(file_name, 'r') as f:
        for i in f.read().split("\n"):
            if len(i) != 0:
                t_s = int(i.split()[2]) # アノテーションの先頭
                t_e = int(i.split()[3]) # アノテーションの末尾
                tag_vocab[t_s] = i.split()[1]
                tag_number.append(t_s)
                tag_number.append(t_e)
    tag_number = sorted(tag_number)
    for k, v in sorted(tag_vocab.items()):
        tag_label.append(v)
    
    # 原文データ用意
    mt = MeCab.Tagger('-Ochasen')
    result = mt.parseToNode("")
    
    file_name = file_name.replace("ann","txt")
     
    word_count = -1 # 単語サイズ
    n = 0 # タグ番号のインデックス
    p = -1 # アノテーションのラベルのインデックス
    
    features_set = []

    with open(file_name, 'r') as f:
        data_text = f.read() # ファイル読み込み
        data_text = data_text.replace("\n",",") # mecabの改行コードが消える部分を補完(「\n」→「,」)
        result = mt.parseToNode(data_text) # 形態素解析
        
        while result:
            
            feature_list = result.feature.split(',')
            
            # 先頭処理
            if feature_list[0] == 'BOS/EOS':
                result = result.next
                continue                        

            features = [] # 特徴            
            features.append(result.surface) # 単語特徴            
            for feature in feature_list: # 形態素特徴
                features.append(feature)
            
            word_count += len(result.surface) # 単語サイズ
            
            # 文末処理
            if n >= len(tag_number):
                features.append("O")
                features_set.append(features)
                result = result.next
                continue 
          
            # タグ処理
            if n%2 == 0:
                if tag_number[n] <= word_count:
                    p += 1
                    features.append("B-{0}".format(tag_label[p]))
                    n += 1
                else:
                    features.append("O")
            elif n%2 != 0:    
                if tag_number[n] <=  word_count:
                    features.append("O")
                    n += 1
                else:
                    features.append("I-{0}".format(tag_label[p]))

            features_set.append(features)
            result = result.next

    return features_set

In [2]:
# データ読み込み
file_info = "/Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/RS_data_fixtxt/rsdata"
data_size = 215 # データサイズ

# 書き出しファイル準備
output_f = open("corpus_5w1hs_mecab.txt", "w")

skip_file = [107,129,125,151,173,195]
data_sum = 0
for file_id in range(1,data_size+1): 
    file_txt = "{0}{1}.ann".format(file_info,file_id)
    
    if file_id in skip_file:
        continue
    data_sum += 1
    print(file_id,file_txt)
    output_line = ""
    for line in get_iobcorpus(file_txt):
        if line[0] == ",": # mecabの改行コードが消える部分を補完 ( 「,」 → 「 」)
            output_f.writelines("\n") 
        else:
            output_line = "\t".join(line)
            output_f.writelines(output_line+"\n")
    output_f.writelines("\n") # 5w1h用の改行コマンド
        
output_f.close()
print(data_sum)

1 /Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/RS_data_fixtxt/rsdata1.ann
2 /Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/RS_data_fixtxt/rsdata2.ann
3 /Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/RS_data_fixtxt/rsdata3.ann
4 /Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/RS_data_fixtxt/rsdata4.ann
5 /Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/RS_data_fixtxt/rsdata5.ann
6 /Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/RS_data_fixtxt/rsdata6.ann
7 /Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/RS_data_fixtxt/rsdata7.ann
8 /Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/RS_data_fixtxt/rsdata8.ann
9 /Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/RS_data_fixtxt/rsdata9.ann
10 /Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/RS_data_fixtxt/rsdata10.ann
11 /User